In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.edge.service import Service as EdgeService
# from selenium.webdriver.chrome.options import Options as ChromeOptions
# from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import os
import sys
import time
import logging
import pandas as pd
from openpyxl import Workbook, load_workbook
from bs4 import BeautifulSoup

# 导入tkinter用于文件选择对话框
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

# 获取资源文件路径的辅助函数
def get_resource_path(relative_path):
    """获取资源文件的绝对路径，兼容开发环境和打包后的环境"""
    if hasattr(sys, '_MEIPASS'):
        # 打包后的环境
        return os.path.join(sys._MEIPASS, relative_path)
    else:
        # 开发环境
        return os.path.join(os.path.abspath("."), relative_path)

# 配置日志
log_file_path = "static/complete_example/crawler.log"
# 获取日志文件所在目录
log_dir = os.path.dirname(log_file_path)
# 如果目录不存在，则创建
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file_path),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class AdvancedCrawler:
    def __init__(self, cookie_file_path="static/complete_example/cookies.json", headless=False):
        """初始化爬虫
        Args:
            cookie_file_path: Cookies保存路径
            headless: 是否使用无头模式
        """
        self.cookie_file_path = cookie_file_path
        self.headless = headless
        self.driver = None
        self.setup_browser_options()

    def setup_browser_options(self):
        """设置浏览器选项"""
        self.options = Options()
        # 基本配置
        self.options.add_argument("--disable-notifications")
        self.options.add_argument("--disable-infobars")
        self.options.add_argument("--start-maximized")
        # 禁用图片加载，提升速度
        # self.options.add_argument("--blink-settings=imagesEnabled=false")
        # 设置用户代理
        self.options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        # 无头模式
        if self.headless:
            self.options.add_argument("--headless")
            self.options.add_argument("--window-size=1920,1080")
            self.options.add_argument("--disable-gpu")

    def start_browser(self, chrome_driver_path=None):
        """启动浏览器
        Args: chrome_driver_path: 本地ChromeDriver路径（可选），如果提供则使用本地驱动，否则尝试自动下载
        Returns: bool: 浏览器启动是否成功
        """
        try:
            logger.info("正在启动浏览器...")

            # 1. 优先使用提供的ChromeDriver路径
            if chrome_driver_path:
                if os.path.exists(chrome_driver_path):
                    logger.info(f"使用打包的ChromeDriver: {chrome_driver_path}")
                    service = Service(chrome_driver_path)
            else:
                # 3. 最后尝试自动下载ChromeDriver
                logger.info("尝试自动下载ChromeDriver...")
                try:
                    from webdriver_manager.chrome import ChromeDriverManager
                    service = Service(ChromeDriverManager().install())
                    #from webdriver_manager.microsoft import EdgeDriverManager
                    #service = Service(EdgeDriverManager().install())
                except Exception as e:
                    logger.error(f"自动下载ChromeDriver失败: {str(e)}")
                    raise
            self.driver = webdriver.Chrome(
                service=service,
                options=self.options
            )
            logger.info("浏览器启动成功")
            return True
        except Exception as e:
            logger.error(f"浏览器启动失败: {str(e)}")
            logger.info("请确保ChromeDriver可用或尝试手动指定路径")
            return False

    def navigate_to(self, url, wait_for_element=None, timeout=10):
        """导航到指定URL并可选择等待特定元素加载
        Args:
            url: 目标URL
            wait_for_element: 等待加载的元素选择器
            timeout: 等待超时时间（秒）
        Returns:
            bool: 是否导航成功
        """
        try:
            logger.info(f"正在访问: {url}")
            self.driver.get(url)
            # 如果指定了等待元素，则等待元素加载
            if wait_for_element:
                WebDriverWait(self.driver, timeout).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, wait_for_element))
                )
                logger.info(f"元素 {wait_for_element} 已加载")
            else:
                # 否则简单等待几秒
                time.sleep(3)
            return True
        except Exception as e:
            logger.error(f"导航失败: {str(e)}")
            return False

    def save_cookies(self):
        """保存当前会话的Cookies
        Returns:
            bool: 是否保存成功
        """
        try:
            if not self.driver:
                logger.error("无法保存Cookies，浏览器未启动")
                return False
            cookies = self.driver.get_cookies()
            # 确保目录存在
            cookie_dir = os.path.dirname(self.cookie_file_path)
            if cookie_dir and not os.path.exists(cookie_dir):
                os.makedirs(cookie_dir)
            with open(self.cookie_file_path, 'w', encoding='utf-8') as f:
                json.dump(cookies, f, ensure_ascii=False, indent=4)
            logger.info(f"成功保存 {len(cookies)} 个Cookies到 {os.path.abspath(self.cookie_file_path)}")
            return True
        except Exception as e:
            logger.error(f"保存Cookies失败: {str(e)}")
            return False

    def load_cookies(self, url=None):
        """加载Cookies到当前会话
        Args: url: 如果提供，会先访问该URL再加载Cookies
        Returns: bool: 是否加载成功
        """
        try:
            if not self.driver:
                logger.error("无法加载Cookies，浏览器未启动")
                return False
            # 如果提供了URL，先访问
            if url:
                self.navigate_to(url)
            # 检查文件是否存在
            if not os.path.exists(self.cookie_file_path):
                logger.error(f"Cookies文件不存在: {os.path.abspath(self.cookie_file_path)}")
                return False
            # 读取Cookies
            with open(self.cookie_file_path, 'r', encoding='utf-8') as f:
                cookies = json.load(f)
            # 清除现有Cookies
            self.driver.delete_all_cookies()
            # 添加Cookies
            success_count = 0
            for cookie in cookies:
                try:
                    # 处理可能的缺失字段
                    if 'expiry' in cookie and isinstance(cookie['expiry'], float):
                        cookie['expiry'] = int(cookie['expiry'])
                    self.driver.add_cookie(cookie)
                    success_count += 1
                except Exception as e:
                    logger.warning(f"添加Cookie失败: {str(e)}，跳过该Cookie")
            # 刷新页面以应用Cookies
            self.driver.refresh()
            logger.info(f"成功加载 {success_count}/{len(cookies)} 个Cookies")
            return True
        except Exception as e:
            logger.error(f"加载Cookies失败: {str(e)}")
            return False

    def take_screenshot(self, filename="screenshot.png"):
        """截取当前页面的屏幕截图
        Args: filename: 保存的文件名
        Returns: bool: 是否截图成功
        """
        try:
            if not self.driver:
                logger.error("无法截图，浏览器未启动")
                return False
            self.driver.save_screenshot(filename)
            logger.info(f"截图已保存到 {os.path.abspath(filename)}")
            return True
        except Exception as e:
            logger.error(f"截图失败: {str(e)}")
            return False

    def close_browser(self):
        """关闭浏览器"""
        try:
            if self.driver:
                logger.info("正在关闭浏览器...")
                self.driver.quit()
                self.driver = None
                logger.info("浏览器已关闭")
        except Exception as e:
            logger.error(f"关闭浏览器失败: {str(e)}")


In [6]:
input_file_path = 'static/complete_example/公司列表.xlsx'
# 1. 读取公司列表
logger.info(f"已选择输入文件: {input_file_path}")
print(f"已选择输入文件: {input_file_path}")
df = pd.read_excel(input_file_path)
company_names = df['工商全称'].dropna().tolist()
print(f'公司列表共 {len(company_names)} 家')

2025-12-21 21:02:07,732 - INFO - 已选择输入文件: static/complete_example/公司列表.xlsx


已选择输入文件: static/complete_example/公司列表.xlsx
公司列表共 1968 家


In [5]:
"""演示完整的爬虫工作流程"""
# --------------------------
# 1. 文件选择阶段
# --------------------------
logger.info("===== 开始演示完整爬虫流程 =====")
logger.info("\n----- 文件选择阶段 -----")
# 创建tkinter窗口（隐藏）
root = tk.Tk()
root.withdraw()
# 弹出文件选择对话框
# input_file_path = filedialog.askopenfilename(
#     title="选择公司列表文件",
#     filetypes=[("Excel文件", "*.xlsx"), ("所有文件", "*.*")]
# )
input_file_path = 'static/complete_example/2022年融过资 表1.xlsx'
# 1. 读取公司列表
logger.info(f"已选择输入文件: {input_file_path}")
print(f"已选择输入文件: {input_file_path}")
df = pd.read_excel(input_file_path)
company_names = df['工商全称'].tolist()
print(f'公司列表共 {len(company_names)} 家')

# 网站URL示例（使用百度作为演示）
target_url = "https://xunkebao.baidu.com"

# Step1：首次访问并保存Cookies
logger.info("\n----- 第一阶段：首次访问并保存Cookies -----")
crawler = AdvancedCrawler()
if crawler.start_browser():
    crawler.navigate_to(target_url) # 1-1: 访问网站
    input("请在浏览器中执行任何需要的操作（如登录），完成后按Enter键继续...") # 1-2: 提示用户可以手动登录（如果需要）
    crawler.take_screenshot("static/complete_example/before_save_cookies.png") # 1-3: 截图记录当前状态
    crawler.save_cookies() # 1-4: 保存Cookies
    # crawler.close_browser() # 1-5: 关闭浏览器

# Step2：使用保存的Cookies重新访问
new_crawler = crawler


2025-12-21 20:50:42,820 - INFO - ===== 开始演示完整爬虫流程 =====
2025-12-21 20:50:42,822 - INFO - 
----- 文件选择阶段 -----
2025-12-21 20:50:43,304 - INFO - 已选择输入文件: static/complete_example/2022年融过资 表1.xlsx


已选择输入文件: static/complete_example/2022年融过资 表1.xlsx


2025-12-21 20:50:44,174 - INFO - 
----- 第一阶段：首次访问并保存Cookies -----
2025-12-21 20:50:44,174 - INFO - 正在启动浏览器...
2025-12-21 20:50:44,175 - INFO - 尝试自动下载ChromeDriver...
2025-12-21 20:50:44,210 - INFO - ====== WebDriver manager ======


公司列表共 5000 家


2025-12-21 20:50:46,427 - INFO - Get LATEST chromedriver version for google-chrome
2025-12-21 20:50:48,398 - INFO - Get LATEST chromedriver version for google-chrome
2025-12-21 20:50:49,157 - INFO - Driver [C:\Users\xuezhihuan\.wdm\drivers\chromedriver\win64\142.0.7444.175\chromedriver.exe] found in cache
2025-12-21 20:50:50,697 - INFO - 浏览器启动成功
2025-12-21 20:50:50,698 - INFO - 正在访问: https://xunkebao.baidu.com
2025-12-21 20:54:26,472 - INFO - 截图已保存到 C:\codes\chrome_crawler\static\complete_example\before_save_cookies.png
2025-12-21 20:54:26,485 - INFO - 成功保存 22 个Cookies到 C:\codes\chrome_crawler\static\complete_example\cookies.json
2025-12-21 20:54:26,487 - INFO - 已发现产出文件，将直接读取
2025-12-21 20:54:26,508 - INFO - 已存在 5 家公司
2025-12-21 20:54:26,509 - INFO - 公司 上海友升铝业股份有限公司 已存在，跳过
2025-12-21 20:54:26,509 - INFO - 公司 宽腾（河南）医疗技术有限公司 已存在，跳过
2025-12-21 20:54:26,510 - INFO - 公司 北京未尔锐创科技股份有限公司 已存在，跳过
2025-12-21 20:54:26,510 - INFO - 公司 南京喜悦科技股份有限公司 已存在，跳过
2025-12-21 20:54:26,511 - INFO - 公司 天津芯森电子科技

产出文件路径: static/complete_example/output.xlsx


2025-12-21 20:54:32,055 - INFO - 开始处理公司 7/5000: 北京海德利森科技有限公司
2025-12-21 20:54:33,251 - ERROR - 顺序爬取公司信息失败: Message: element click intercepted: Element <button class="el-button el-button--primary search-btn" type="button" data-v-8a58691e="">...</button> is not clickable at point (947, 69). Other element would receive the click: <div class="top" data-v-35b2afaa="">...</div>
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xf34103
	0xf34144
	0xd3e71d
	0xd90580
	0xd8e8e3
	0xd8c387
	0xd8b5fd
	0xd7fd65
	0xdac90c
	0xd7f7c4
	0xdacac4
	0xdcee17
	0xdac706
	0xd7da30
	0xd7ed54
	0x11a57b4
	0x11a098a
	0xf5c392
	0xf4c4c8
	0xf5324d
	0xf3c478
	0xf3c63c
	0xf267ca
	0x76d45d49
	0x77bdd6db
	0x77bdd661

2025-12-21 20:54:43,254 - INFO - 
===== 爬虫流程演示完成 =====


In [37]:
 search_res_num = new_crawler.driver.find_element(By.CSS_SELECTOR, 'div.middle-bar > div.info > span:nth-child(1) > em')
 search_res_num.text

'0'

In [38]:
# Step3: 顺序爬取公司信息
try:
    # 2. 创建结果文件（如存在则直接读取）
    # output_file_path = os.path.join(os.path.dirname(__file__),'static','complete_example','output.xlsx')
    output_file_path = 'static/complete_example/output.xlsx'
    print(f'产出文件路径: {output_file_path}')
    if not os.path.exists(output_file_path):
        logger.info('未发现产出文件，将新建')
        wb = Workbook()
        ws_phone = wb.active
        ws_phone.title = "手机"
        ws_tel = wb.create_sheet("座机") # 创建第二个sheet "邮箱"
        ws_email = wb.create_sheet("邮箱") # 创建第二个sheet "邮箱"
        ws_qq = wb.create_sheet("QQ") # 创建第三个sheet "QQ"
        wb.save(output_file_path) # 保存工作簿
        existed_company_names = []
    else:
        logger.info('已发现产出文件，将直接读取')
        phone_df = pd.read_excel(output_file_path, sheet_name='手机', header=None)
        if len(phone_df)==0:
            existed_company_names = []
        else:
            existed_company_names = set([v.split('|')[0] for v in phone_df[0]])
        logger.info(f'已存在 {len(existed_company_names)} 家公司')

    # 3. 顺序遍历爬取
    for i, company_name in enumerate(company_names):
        if company_name in existed_company_names:
            logger.info(f'公司 {company_name} 已存在，跳过')
            continue
        logger.info(f'开始处理公司 {i+1}/{len(company_names)}: {company_name}')
        # 3-1. 搜索公司
        search_input = new_crawler.driver.find_element(By.CSS_SELECTOR, "div.search-input-wrap > section > div > div > div > div > input")
        search_btn = new_crawler.driver.find_element(By.CSS_SELECTOR, "div.search-input-wrap > section > div > button.el-button.el-button--primary.search-btn")

        search_input.clear()
        search_input.send_keys(company_name)
        search_btn.click()
        # 3-2. 等待搜索结果加载
        time.sleep(3)

        # 3-3. 截图记录当前状态, 点开公司详情页
        search_res_num = new_crawler.driver.find_element(By.CSS_SELECTOR, 'div.middle-bar > div.info > span:nth-child(1) > em')
        if search_res_num.text=='0':
            continue
        # new_crawler.take_screenshot(f"static/complete_example/{company_name.replace(' ','_')}.png")
        company_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "h6.company-name")
        is_find_btn=0
        for btn in company_btns:
            if btn.text.strip()==company_name:
                is_find_btn=1
                btn.click()
                time.sleep(2)
                break
        if is_find_btn==0:
            continue

        # 3-3 点击公司详情页的【立即查看】按钮
        check_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "div.check > button")
        for btn in check_btns:
            if btn.text.strip() == "立即查看":
                # 滑动到对应按钮可见
                new_crawler.driver.execute_script("arguments[0].scrollIntoView(false);", btn)
                time.sleep(1)
                # 确保按钮可点击，再点击
                WebDriverWait(new_crawler.driver, 2).until( EC.element_to_be_clickable(btn) )
                time.sleep(1)
                btn.click()
            break
        while True:
            time.sleep(2)
            check_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "div.check > button")
            if check_btns[0].text == "空错号检测":
                break
                # if new_crawler.driver.find_element(By.CSS_SELECTOR, "button.el-dialog__headerbtn"):
                #     logger.info(f'点击【关闭】按钮成功，但无法顺利展示，提前跳出爬取逻辑')

        # 3-4. 保存网页源代码，解析公司联系信息
        page_source = new_crawler.driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        modules = soup.select("#app > div > div.el-overlay.popMenu-box > div > section > div.container > div.coding-box > div > div")
        result_dict = {}
        for m in modules:
            title_text = m.select_one('div.list-item-title').get_text()
            title_class = title_text.split(' ')[0]
            item_text_lst = [t.get_text(separator='|') for t in m.select('div.contact-item')]
            result_dict[title_class] = [f'{company_name}|{title_text}|{i}' for i in item_text_lst]
            # print(f'{title_class}: {result_dict[title_class]}')
        # 3-5. 保存公司信息到结果文件
        wb = load_workbook(output_file_path)
        for n in ('手机', '座机', '邮箱', 'QQ'):
            ws = wb[n]
            for item in result_dict.get(n,[]):
                if isinstance(item, str):
                    ws.append([item])
        wb.save(output_file_path)
        time.sleep(2)
        # 3-6. 返回搜索页
        backhome_btn = new_crawler.driver.find_element(By.CSS_SELECTOR, "div.el-overlay.popMenu-box > div > section > div.pack-up")
        backhome_btn.click()
        time.sleep(1)

except Exception as e:
    logger.error(f"顺序爬取公司信息失败: {str(e)}")
    time.sleep(10)

logger.info("\n===== 爬虫流程演示完成 =====")

2025-12-21 22:46:04,578 - INFO - 已发现产出文件，将直接读取
2025-12-21 22:46:04,644 - INFO - 已存在 167 家公司
2025-12-21 22:46:04,646 - INFO - 公司 成都飞亚航空设备应用研究所有限公司 已存在，跳过
2025-12-21 22:46:04,647 - INFO - 公司 深圳市威湃创新科技有限公司 已存在，跳过
2025-12-21 22:46:04,648 - INFO - 公司 深圳创智芯联科技股份有限公司 已存在，跳过
2025-12-21 22:46:04,648 - INFO - 公司 深圳市品罗创新实业有限公司 已存在，跳过
2025-12-21 22:46:04,649 - INFO - 公司 深圳市路远智能装备有限公司 已存在，跳过
2025-12-21 22:46:04,650 - INFO - 公司 上海果纳半导体技术有限公司 已存在，跳过
2025-12-21 22:46:04,650 - INFO - 公司 武汉长利新材料科技股份有限公司 已存在，跳过
2025-12-21 22:46:04,652 - INFO - 公司 上海时的科技有限公司 已存在，跳过
2025-12-21 22:46:04,653 - INFO - 公司 深圳屹艮科技有限公司 已存在，跳过
2025-12-21 22:46:04,654 - INFO - 公司 新石器慧通（北京）科技有限公司 已存在，跳过
2025-12-21 22:46:04,655 - INFO - 公司 合肥欣奕华智能机器股份有限公司 已存在，跳过
2025-12-21 22:46:04,656 - INFO - 公司 深圳中智卫安机器人技术有限公司 已存在，跳过
2025-12-21 22:46:04,656 - INFO - 公司 杭州艾诺半导体有限公司 已存在，跳过
2025-12-21 22:46:04,657 - INFO - 公司 微见智能封装技术（深圳）有限公司 已存在，跳过
2025-12-21 22:46:04,658 - INFO - 公司 西安华泰半导体科技有限公司 已存在，跳过
2025-12-21 22:46:04,658 - INFO - 公司 珠海镓未来科技有限

产出文件路径: static/complete_example/output.xlsx


2025-12-21 22:46:08,070 - INFO - 公司 苏州倍丰智能科技有限公司 已存在，跳过
2025-12-21 22:46:08,071 - INFO - 公司 东方空间（山东）科技有限公司 已存在，跳过
2025-12-21 22:46:08,072 - INFO - 公司 浙江天洁磁性材料股份有限公司 已存在，跳过
2025-12-21 22:46:08,072 - INFO - 公司 上海维安电子股份有限公司 已存在，跳过
2025-12-21 22:46:08,073 - INFO - 公司 艾可萨科技（成都）有限公司 已存在，跳过
2025-12-21 22:46:08,073 - INFO - 公司 深圳市深光谷科技有限公司 已存在，跳过
2025-12-21 22:46:08,074 - INFO - 公司 芯行纪科技有限公司 已存在，跳过
2025-12-21 22:46:08,074 - INFO - 公司 广州海工船舶设备有限公司 已存在，跳过
2025-12-21 22:46:08,076 - INFO - 公司 卡门哈斯激光科技（苏州）有限公司 已存在，跳过
2025-12-21 22:46:08,076 - INFO - 公司 中航迈特增材科技（北京）有限公司 已存在，跳过
2025-12-21 22:46:08,076 - INFO - 公司 南京能利芯科技有限公司 已存在，跳过
2025-12-21 22:46:08,078 - INFO - 公司 北京物芯科技有限责任公司 已存在，跳过
2025-12-21 22:46:08,079 - INFO - 公司 无锡国芯微电子技术股份有限公司 已存在，跳过
2025-12-21 22:46:08,079 - INFO - 公司 广东盈骅新材料科技有限公司 已存在，跳过
2025-12-21 22:46:08,080 - INFO - 公司 安徽科幂仪器有限公司 已存在，跳过
2025-12-21 22:46:08,080 - INFO - 公司 安徽凯木金电子科技有限公司 已存在，跳过
2025-12-21 22:46:08,081 - INFO - 公司 合肥芯谷微电子股份有限公司 已存在，跳过
2025-12-21 22:46:08,081 - INFO - 公司

KeyboardInterrupt: 

In [31]:
company_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "h6.company-name")
company_btns[0].text.strip()

'南京金邦动力科技有限公司工会委员会'

In [17]:
# 3-3 点击公司详情页的【立即查看】按钮
# check_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "div.check > button")
for btn in check_btns:
    if btn.text == "立即查看":
        # 滑动到对应按钮可见
        new_crawler.driver.execute_script("arguments[0].scrollIntoView(false);", btn)
        time.sleep(0.3)
        # 确保按钮可点击，再点击
        WebDriverWait(new_crawler.driver, 2).until( EC.element_to_be_clickable(btn) )
        btn.click()
        time.sleep(2)
        if new_crawler.driver.find_element(By.CSS_SELECTOR, "button.el-dialog__headerbtn"):
            logger.info(f'点击【关闭】按钮成功，但无法顺利展示，提前跳出爬取逻辑')
    break


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"button.el-dialog__headerbtn"}
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xf34103
	0xf34144
	0xd3e71d
	0xd8a03d
	0xd8a41b
	0xdd17f2
	0xdac954
	0xdcee17
	0xdac706
	0xd7da30
	0xd7ed54
	0x11a57b4
	0x11a098a
	0xf5c392
	0xf4c4c8
	0xf5324d
	0xf3c478
	0xf3c63c
	0xf267ca
	0x76d45d49
	0x77bdd6db
	0x77bdd661


In [3]:
# 3-3 点击公司详情页的【立即查看】按钮
check_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "div.check > button")
for btn in check_btns:
    if btn.text == "点击查看":
        # 滑动到对应按钮可见
        new_crawler.driver.execute_script("arguments[0].scrollIntoView(false);", btn)
        time.sleep(0.3)
        # 确保按钮可点击，再点击
        WebDriverWait(new_crawler.driver, 2).until( EC.element_to_be_clickable(btn) )
        btn.click()
        time.sleep(2)
        if new_crawler.driver.find_element(By.CSS_SELECTOR, "button.el-dialog__headerbtn"):
            logger.info(f'点击【关闭】按钮成功，但无法顺利展示，提前跳出爬取逻辑')
    break
# 3-4. 保存网页源代码，解析公司联系信息
page_source = new_crawler.driver.page_source
soup = BeautifulSoup(page_source, "html.parser")
modules = soup.select("#app > div > div.el-overlay.popMenu-box > div > section > div.container > div.coding-box > div > div")
result_dict = {}
for m in modules:
    title_text = m.select_one('div.list-item-title').get_text()
    title_class = title_text.split(' ')[0]
    item_text_lst = [t.get_text(separator='|') for t in m.select('div.contact-item')]
    result_dict[title_class] = [f'{company_name}|{title_text}|{i}' for i in item_text_lst]
    # print(f'{title_class}: {result_dict[title_class]}')
# 3-5. 保存公司信息到结果文件
wb = load_workbook(output_file_path)
for n in ('手机', '座机', '邮箱', 'QQ'):
    ws = wb[n]
    for item in result_dict.get(n,[]):
        if isinstance(item, str):
            ws.append([item])
wb.save(output_file_path)
time.sleep(2)
# 3-6. 返回搜索页
backhome_btn = new_crawler.driver.find_element(By.CSS_SELECTOR, "div.el-overlay.popMenu-box > div > section > div.pack-up")
backhome_btn.click()
time.sleep(1)

In [26]:
page_source = new_crawler.driver.page_source
page_source

'<html lang="en"><head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="keywords" content="2022年电话，最新联系方式查询，番番寻客宝">\n    <meta name="description" content="番番寻客宝为您提供的电话、邮箱、QQ、联系人信息、联系人职位预测、号码归属地、号码检测信息、关联和相似企业联系方式,让您更轻松地了解2022年最新的联系方式,查询最新、最全、最优质的公司联系方式就来番番寻客宝!">\n    <meta name="baidu-site-verification" content="code-4j0GpMr4rV">\n    <meta name="msvalidate.01" content="9241D0B13A807876AA216B715A08C258">\n    <title>番番寻客宝</title>\n    <link rel="preconnect" href="https://fe-aff-su.cdn.bcebos.com" crossorigin="">\n    <link rel="manifest" href="https://fe-aff.cdn.bcebos.com/biz-crm-fe-xkb-pc/online/manifest.json">\n    <script async="" src="https://aiff.cdn.bcebos.com/sensors%2Fonline%2Fsa-sdk-javascript-1.14.24%2Fsensorsdata.min.js" charset="UTF-8"></script><script src="https://hm.baidu.com/hm.js?18ca88c840f4f94ef856298c2c8435a9"></script><script>\n        if (location.host === \'aifanfan.baidu.com\') {\n         

In [ ]:
# 3-3 点击公司详情页的【立即查看】按钮
check_btns = new_crawler.driver.find_elements(By.CSS_SELECTOR, "div.check > button")
for btn in check_btns:
    # 滑动到对应按钮可见
    new_crawler.driver.execute_script("arguments[0].scrollIntoView(false);", btn)
    time.sleep(2)
    # 确保按钮可点击，再点击
    WebDriverWait(new_crawler.driver, 2).until( EC.element_to_be_clickable(btn) )
    btn.click()
    time.sleep(2)

ElementClickInterceptedException: Message: element click intercepted: Element <button class="el-button" type="button" data-v-51c08b57="" style="--el-button-bg-color: #EF1F1F; --el-button-text-color: var(--el-color-white); --el-button-border-color: #EF1F1F; --el-button-hover-bg-color: rgb(244, 98, 98); --el-button-hover-text-color: var(--el-color-white); --el-button-hover-border-color: rgb(244, 98, 98); --el-button-active-bg-color: rgb(195, 29, 29); --el-button-active-border-color: rgb(195, 29, 29); color: white; font-size: 12px; padding: 10px 20px;">...</button> is not clickable at point (396, 16). Other element would receive the click: <div class="header-box" data-v-35b2afaa="">...</div>
  (Session info: chrome=142.0.7444.176); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#elementclickinterceptedexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x5b4103
	0x5b4144
	0x3be71d
	0x410580
	0x4

In [27]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")
modules = soup.select("#app > div > div.el-overlay.popMenu-box > div > section > div.container > div.coding-box > div > div")
modules

[<div class="list-item" data-v-4c9e58b2=""><div class="list-item-title" data-text="手机" data-v-4c9e58b2="">手机 <span data-v-4c9e58b2="">33</span><strong class="link" data-v-4c9e58b2="">隐藏</strong></div><div class="contact-item" data-type="0" data-v-4c9e58b2="" style="margin-right: 12px;"><div class="dis-start-desc not-dis" data-v-4c9e58b2="" data-v-51c08b57=""><div class="p" data-v-51c08b57=""><div class="name-con" data-v-51c08b57=""><div class="name-img" data-v-51c08b57="">-</div><span class="text f-1" data-v-51c08b57=""><span class="text" data-v-51c08b57="">1582</span><span class="text dim" data-v-51c08b57="">****</span><span class="text" data-v-51c08b57="">704</span></span></div></div><div data-v-51c08b57="" style="margin-top: 11px; margin-bottom: 7px;"><!-- --><!-- --><span class="tag default" data-v-51c08b57="">30天内更新</span></div><div class="ame" data-v-51c08b57=""><span data-v-51c08b57="">联系人：</span><span class="dim" data-v-51c08b57="">未知</span></div><!-- --><div class="ame f-1" da

In [53]:
result_lst = []
for m in modules:
    title_text = m.select_one('div.list-item-title').get_text()
    print(title_text)
    item_text_lst = [t.get_text(separator='|') for t in m.select('div.contact-item')]
    print(item_text_lst)
    result_lst += [f'{company_name}|{title_text}|{i}' for i in item_text_lst]

手机 33隐藏
['-|1582|****|704|30天内更新|联系人：|未知|来源：| 立即查看 ', '张|1365|****|440|30天内更新|联系人：|张**|来源：|亿**|阿**| 立即查看 ', '龚|1376|****|334|30天内更新|联系人：|龚**|来源：|一**|汽车***|铝**| 立即查看 ', '李|1590|****|392|30天内更新|联系人：|李**|来源：|八方***|垂直***| 立即查看 ', '联|1592|****|670|30天内更新|联系人：|联**|来源：|企**|一呼**|102****|首**| 立即查看 ', '罗|1399|****|984|30天内更新|联系人：|罗**|来源：|利酷***|智能**| 立即查看 ', '杨|1305|****|873|30天内更新|联系人：|杨**|来源：|马可***|中国***|满分***| 立即查看 ', '龚|1352|****|307|30天内更新|联系人：|龚**|来源：|中国***| 立即查看 ', '杨|1356|****|769|30天内更新|联系人：|杨**|来源：|际**|马可***| 立即查看 ', '-|1330|****|397|30天内更新|联系人：|未知|来源：|新**| 立即查看 ', '罗|1388|****|512|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1390|****|657|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1314|****|778|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1339|****|584|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1877|****|005|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1580|****|318|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1381|****|866|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1380|****|615|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 ', '罗|1304|****|602|30天内

In [41]:
backhome_btn = new_crawler.driver.find_element(By.CSS_SELECTOR, "div.el-overlay.popMenu-box > div > section > div.pack-up")
backhome_btn.click()

In [57]:
result_lst[0]

'宽腾（河南）医疗技术有限公司|手机 33隐藏|-|1582|****|704|30天内更新|联系人：|未知|来源：| 立即查看 '

In [59]:
import csv
output_file_path = "./static/complete_example/output.csv"
with open(output_file_path, 'a', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    for row in result_lst:
        writer.writerow(row.split('|'))
        print(f"已写入多字段行: {row}")

已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|-|1582|****|704|30天内更新|联系人：|未知|来源：| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|张|1365|****|440|30天内更新|联系人：|张**|来源：|亿**|阿**| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|龚|1376|****|334|30天内更新|联系人：|龚**|来源：|一**|汽车***|铝**| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|李|1590|****|392|30天内更新|联系人：|李**|来源：|八方***|垂直***| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|联|1592|****|670|30天内更新|联系人：|联**|来源：|企**|一呼**|102****|首**| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|罗|1399|****|984|30天内更新|联系人：|罗**|来源：|利酷***|智能**| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|杨|1305|****|873|30天内更新|联系人：|杨**|来源：|马可***|中国***|满分***| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|龚|1352|****|307|30天内更新|联系人：|龚**|来源：|中国***| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|杨|1356|****|769|30天内更新|联系人：|杨**|来源：|际**|马可***| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|-|1330|****|397|30天内更新|联系人：|未知|来源：|新**| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|罗|1388|****|512|30天内更新|联系人：|罗**|来源：|智能**| 立即查看 
已写入多字段行: 宽腾（河南）医疗技术有限公司|手机 33隐藏|罗|1390|****|657|30天内更新|联系人：|罗**|来源：|

In [58]:
row

'宽腾（河南）医疗技术有限公司|QQ 2|田|84|****|88|联系人：|田**|来源：|阿德***| 立即查看 '